In [45]:
import sys,os
import pandas as pd
from decouple import Config, RepositoryEnv
import random

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

import locale
from datetime import datetime, timedelta


locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

current_directory = os.getcwd()
sys.path.insert(0,f'{current_directory}/../src/')
sys.path.insert(0,f'{current_directory}/../chromedriver-mac-x64/chromedriver')

In [46]:
env_config = Config(RepositoryEnv('./.env'))
email = env_config.get('EMAIL')
password = env_config.get('PASSWORD')

In [49]:
def get_playtomic_schedule(email, password) -> pd.DataFrame:
    
    options = webdriver.ChromeOptions()
    #options.add_argument("--headless=new")
    options.add_argument("--window-size=1920,1080")
    options.add_argument('--no-sandbox')
    options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent
    
    driver = webdriver.Chrome(options=options)
    driver.implicitly_wait(10)
    wait = WebDriverWait(driver, 20)

    try:
        driver.get('https://manager.playtomic.io/auth/login')
        
        driver.execute_script("""
        function hideCookies() {
            const selectors = [
                '#usercentrics-cmp-ui',
                'div[id*="cookie"]', 
                'div[class*="cookie"]',
                'iframe[title*="cookie"]',
                'aside[aria-label*="cookie"]'
            ];
            
            selectors.forEach(selector => {
                document.querySelectorAll(selector).forEach(el => el.remove());
            });
            
            document.body.style.overflow = 'auto';
        }
        hideCookies();
        setInterval(hideCookies, 1000);
        """ 
        )
        
        time.sleep(random.uniform(5,10))
        
        wait.until(EC.presence_of_element_located((By.NAME, "email")))

        driver.find_element(By.NAME, "email").send_keys(email)
        driver.find_element(By.NAME, "password").send_keys(password)
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
        
        time.sleep(random.uniform(5,10))
        
        # Espera a que el menú de reservas esté presente
        wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Reservas"))).click()
        
        time.sleep(random.uniform(5,10))

        data, headers = [], []
        for _ in range(6):
            try:
                table = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "table.Tablestyles__Table-gx0hbp-0")))
                
                if not headers:
                    headers = [th.text for th in table.find_elements(By.CSS_SELECTOR, "thead th")]
            
                for row in table.find_elements(By.CSS_SELECTOR, "tbody tr"):
                    data.append([td.text for td in row.find_elements(By.TAG_NAME, "td")])
                    
            except:
                break
            
            try:
                wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Siguiente']"))).click()
                time.sleep(random.uniform(5,10))
            except TimeoutException:
                print("No hay más páginas.")
                break

        reservation_df = pd.DataFrame(data, columns=headers)
        reservation_df = reservation_df[reservation_df.Estado != 'Cancelada']
        
        reservation_df.loc[:,'start_dt'] = pd.to_datetime(reservation_df['Fecha de servicio'], dayfirst=True)
        reservation_df.loc[:,'fecha_reserva'] = reservation_df['start_dt'].dt.date
        reservation_df.loc[:,'hora_inicio'] = reservation_df['start_dt'].dt.strftime('%H:%M')

        dur = reservation_df['Duración'].str.replace('hr', 'h').str.replace(' min', 'm')
        reservation_df.loc[:,'dur_td'] = pd.to_timedelta(dur)
        reservation_df.loc[:,'end_dt'] = reservation_df['start_dt'] + reservation_df['dur_td']
        reservation_df.loc[:,'hora_fin'] = reservation_df['end_dt'].dt.strftime('%H:%M')
        
        return reservation_df[['start_dt','end_dt']]

    except Exception as e:
        print(f"Error durante scraping: {e}")
        raise
    finally:
        driver.quit()

In [50]:
reservation_df = get_playtomic_schedule(email,password)

In [51]:
reservation_df

,start_dt,end_dt
1,2025-07-13 20:00:00,2025-07-13 21:30:00
2,2025-07-13 18:30:00,2025-07-13 19:30:00
3,2025-07-13 17:30:00,2025-07-13 18:30:00
4,2025-07-13 16:00:00,2025-07-13 17:30:00
5,2025-07-13 14:30:00,2025-07-13 15:30:00
6,2025-07-13 13:00:00,2025-07-13 14:00:00
7,2025-07-13 11:30:00,2025-07-13 13:00:00
8,2025-07-13 10:00:00,2025-07-13 11:30:00
9,2025-07-14 20:30:00,2025-07-14 22:00:00
10,2025-07-14 19:00:00,2025-07-14 20:30:00


In [ ]:
def add_playtomic_schedule(email, password, day_reservation, fecha_inicio, match_duration, court_price, court_name) -> None:
    
    def calcular_fecha_fin(fecha_inicio, match_duration):
        fmt = "%H:%M"
        start = datetime.strptime(fecha_inicio, fmt)
        num, unit = match_duration.split()
        dur = timedelta(minutes=int(num)) if unit.startswith('min') else timedelta(hours=int(num))
        end = start + dur
        return end.strftime(fmt)
    
    def click_select_and_choose(selector_control, text):
        ctl = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, selector_control)))
        actions.move_to_element(ctl).click().perform()
        opt = wait.until(EC.element_to_be_clickable(
            (By.XPATH, f"//div[contains(@class,'select__option') and text()='{text}']")))
        opt.click()
        
    def check_is_correct_date(day_reservation,days_to_check=7):
        
        for _ in range(days_to_check):
        
            # Suponiendo que ya tienes driver y wait definidos
            span = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "span.sc-kVhXZc.cueJXp")))
            current_date = span.text
            
            year = datetime.now().year  # por ejemplo, 2025
            current_date_dt = datetime.strptime(f"{year}, {current_date}", f"%Y, %a, %d %b")
            
            if current_date_dt == day_reservation:
                break
            else:
                wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Siguiente']"))).click()
                time.sleep(random.uniform(5,10))
                
    def check_reservation_exists(reservation_df, day_reservation, fecha_inicio, match_duration):
        
        dt_start = pd.Timestamp(f"{day_reservation.date()} {fecha_inicio}")
        dur = pd.to_timedelta(match_duration.replace('min', 'm').replace('hr', 'h'))
        dt_end = dt_start + dur
        
        FILTRO_START = (reservation_df['start_dt'] <= dt_start) & (dt_start < reservation_df['end_dt'])
        FILTRO_END = (reservation_df['start_dt'] < dt_end) & (dt_end <= reservation_df['end_dt'])
        is_any_match = len(reservation_df[FILTRO_START | FILTRO_END]) > 0
        
        return is_any_match
        
    fecha_fin = calcular_fecha_fin(fecha_inicio, match_duration)
    reservation_df = get_playtomic_schedule(email,password)
        
    try: 
        
        options = webdriver.ChromeOptions()
        options.add_argument("--headless=new")
        options.add_argument("--window-size=1920,1080")
        options.add_argument('--no-sandbox')
        options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent
        
        driver = webdriver.Chrome(options=options)
        wait = WebDriverWait(driver, 20)
        actions = ActionChains(driver)
        
        is_any_match = check_reservation_exists(reservation_df, day_reservation, fecha_inicio, match_duration)
        if is_any_match:
            print("La franja ya está ocupada.")
            return
        
        driver.get('https://manager.playtomic.io/auth/login')
        
        driver.execute_script("""
        function hideCookies() {
            const selectors = [
                '#usercentrics-cmp-ui',
                'div[id*="cookie"]', 
                'div[class*="cookie"]',
                'iframe[title*="cookie"]',
                'aside[aria-label*="cookie"]'
            ];
            
            selectors.forEach(selector => {
                document.querySelectorAll(selector).forEach(el => el.remove());
            });
            
            document.body.style.overflow = 'auto';
        }
        hideCookies();
        setInterval(hideCookies, 1000);
        """ 
        )
        
        time.sleep(random.uniform(5,10))
        
        wait.until(EC.presence_of_element_located((By.NAME, "email")))

        driver.find_element(By.NAME, "email").send_keys(email)
        driver.find_element(By.NAME, "password").send_keys(password)
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
        
        check_is_correct_date(day_reservation)

        # Paso: abrir hora de inicio
        wait.until(EC.presence_of_element_located((By.XPATH, f"//tr[@data-time='{fecha_inicio}:00']")))
        
        num, unit = match_duration.split()
        
        if unit.startswith('min') and int(num)<=60:
            fecha_intermedio = calcular_fecha_fin(fecha_inicio, '15 min')
            fecha_destino = calcular_fecha_fin(fecha_inicio, '30 min')
            
            origen = driver.find_element(By.XPATH, f"//tr[@data-time='{fecha_inicio}:00']//td[@class='fc-widget-content']")
            intermedio = driver.find_element(By.XPATH, f"//tr[@data-time='{fecha_intermedio}:00']//td[@class='fc-widget-content']")
            destino = driver.find_element(By.XPATH, f"//tr[@data-time='{fecha_destino}:00']//td[@class='fc-widget-content']")
            
            casillas = [origen,intermedio,destino]
            
            actions = ActionChains(driver)
            # Empieza en la primera casilla
            actions.move_to_element(casillas[0]).click_and_hold()

            # Pasa por las demás casillas con el clic aún pulsado
            for casilla in casillas[1:]:
                actions.move_to_element(casilla)

            # Suelta el clic en la última
            actions.release().perform()
            
        else:
            driver.find_element(By.XPATH, f"//tr[@data-time='{fecha_inicio}:00']//td[@class='fc-widget-content']").click()

        # Selección horas inicio y fin
        click_select_and_choose("#startDate div.select__control", fecha_inicio)
        click_select_and_choose("#endDate div.select__control", fecha_fin)

        # Clic en "Editar precio"
        wait.until(EC.element_to_be_clickable((By.XPATH, "//button[span[text()='Editar']]"))).click()

        # Ingresar precio
        price_input = wait.until(EC.element_to_be_clickable((By.ID, "input-input-1")))
        price_input.clear()
        price_input.send_keys(court_price)
        
        time.sleep(3)

        input_field = wait.until(EC.element_to_be_clickable(
            (By.CSS_SELECTOR, "input[id^='react-select'][id$='-input']")
        ))

        input_field.send_keys(court_name)

        option = wait.until(EC.element_to_be_clickable(
            (By.XPATH, f"//li//div[@class='ListItemContentstyles__CenterGroup-ny9mc5-2 iTqicv' and contains(., '{court_name}')]")
        ))
        option.click()
        
        # Clic en "Crear reserva"
        wait.until(EC.element_to_be_clickable((By.XPATH, "//button[span[text()='Crear']]"))).click()
        
        time.sleep(5)
        
        print('Se ha añadido la reserva al calendario')

    except Exception as e:
        print("Error durante scraping:", e)
        raise
    finally:
        driver.quit()

In [ ]:
day_reservation = datetime(2025, 7, 14)
fecha_inicio = '08:30'
match_duration = '60 min'
court_price = 0
court_name = 'Prueba'

In [53]:
add_playtomic_schedule(email, password, day_reservation, fecha_inicio, match_duration, court_price, court_name)

Error durante scraping: Message: 
Stacktrace:
0   chromedriver                        0x000000010ec3e308 chromedriver + 6148872
1   chromedriver                        0x000000010ec358ba chromedriver + 6113466
2   chromedriver                        0x000000010e6c6e10 chromedriver + 417296
3   chromedriver                        0x000000010e718c94 chromedriver + 752788
4   chromedriver                        0x000000010e718eb1 chromedriver + 753329
5   chromedriver                        0x000000010e768dd4 chromedriver + 1080788
6   chromedriver                        0x000000010e73eced chromedriver + 908525
7   chromedriver                        0x000000010e76618c chromedriver + 1069452
8   chromedriver                        0x000000010e73ea93 chromedriver + 907923
9   chromedriver                        0x000000010e70b0f7 chromedriver + 696567
10  chromedriver                        0x000000010e70bd61 chromedriver + 699745
11  chromedriver                        0x000000010ebfb250 

TimeoutException: Message: 
Stacktrace:
0   chromedriver                        0x000000010ec3e308 chromedriver + 6148872
1   chromedriver                        0x000000010ec358ba chromedriver + 6113466
2   chromedriver                        0x000000010e6c6e10 chromedriver + 417296
3   chromedriver                        0x000000010e718c94 chromedriver + 752788
4   chromedriver                        0x000000010e718eb1 chromedriver + 753329
5   chromedriver                        0x000000010e768dd4 chromedriver + 1080788
6   chromedriver                        0x000000010e73eced chromedriver + 908525
7   chromedriver                        0x000000010e76618c chromedriver + 1069452
8   chromedriver                        0x000000010e73ea93 chromedriver + 907923
9   chromedriver                        0x000000010e70b0f7 chromedriver + 696567
10  chromedriver                        0x000000010e70bd61 chromedriver + 699745
11  chromedriver                        0x000000010ebfb250 chrome

In [ ]:
def drop_playtomic_schedule(email, password, day_reservation, fecha_inicio, fecha_fin, court_name) -> None:
    
    def check_is_correct_date(day_reservation,days_to_check=7):
        
        for _ in range(days_to_check):
        
            # Suponiendo que ya tienes driver y wait definidos
            span = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "span.sc-kVhXZc.cueJXp")))
            current_date = span.text
            
            year = datetime.now().year  # por ejemplo, 2025
            current_date_dt = datetime.strptime(f"{year}, {current_date}", f"%Y, %a, %d %b")
            
            if current_date_dt == day_reservation:
                break
            else:
                wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Siguiente']"))).click()
                time.sleep(random.uniform(5,10))
                
    def check_reservation_exists(reservation_df, day_reservation, fecha_inicio, fecha_fin):
        
        dt_start = pd.Timestamp(f"{day_reservation.date()} {fecha_inicio}")
        dt_end = dt_start = pd.Timestamp(f"{day_reservation.date()} {fecha_fin}")
        
        FILTRO_START = (reservation_df['start_dt'] <= dt_start) & (dt_start < reservation_df['end_dt'])
        FILTRO_END = (reservation_df['start_dt'] < dt_end) & (dt_end <= reservation_df['end_dt'])
        is_any_match = len(reservation_df[FILTRO_START | FILTRO_END]) > 0
        
        return is_any_match
        
    reservation_df = get_playtomic_schedule(email,password)
        
    try: 
        
        options = webdriver.ChromeOptions()
        options.add_argument("--headless=new")
        options.add_argument("--window-size=1920,1080")
        options.add_argument('--no-sandbox')
        options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""') # user agent
        
        driver = webdriver.Chrome(options=options)
        wait = WebDriverWait(driver, 20)
        actions = ActionChains(driver)
        
        is_any_match = check_reservation_exists(reservation_df, day_reservation, fecha_inicio, fecha_fin)
        if not is_any_match:
            print("No existe ninguna reserva para este dia y esta hora")
            return
        
        driver.get('https://manager.playtomic.io/auth/login')
        
        driver.execute_script("""
        function hideCookies() {
            const selectors = [
                '#usercentrics-cmp-ui',
                'div[id*="cookie"]', 
                'div[class*="cookie"]',
                'iframe[title*="cookie"]',
                'aside[aria-label*="cookie"]'
            ];
            
            selectors.forEach(selector => {
                document.querySelectorAll(selector).forEach(el => el.remove());
            });
            
            document.body.style.overflow = 'auto';
        }
        hideCookies();
        setInterval(hideCookies, 1000);
        """ 
        )
        
        time.sleep(random.uniform(5,10))
        
        wait.until(EC.presence_of_element_located((By.NAME, "email")))

        driver.find_element(By.NAME, "email").send_keys(email)
        driver.find_element(By.NAME, "password").send_keys(password)
        wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()
        
        check_is_correct_date(day_reservation)

        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"a.fc-time-grid-event")))
        
        time.sleep(3)
        
        for element in driver.find_elements(By.CSS_SELECTOR, f"a.fc-time-grid-event"):
            if court_name.lower() in str(element.text).lower() and fecha_inicio in element.text and fecha_fin in element.text:
                element.click()
                
        time.sleep(3)
        
        # Clic en "Crear reserva"
        wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@aria-label='Cancelar reserva']"))).click()
        
        time.sleep(3)

        # Clic en "Confirma cancelar reserva"
        wait.until(EC.element_to_be_clickable((By.XPATH, "//button[span[text()='Cancelar reserva']]"))).click()

        time.sleep(3)
        
        print('Se ha eliminado la reserva al calendario')

    except Exception as e:
        print("Error durante scraping:", e)
        raise
    finally:
        driver.quit()

In [43]:
day_reservation = datetime(2025, 7, 8)
fecha_inicio = '10:00'
fecha_fin = '11:00'
court_price = 0
court_name = 'Prueba'

In [44]:
drop_playtomic_schedule(email, password, day_reservation, fecha_inicio, fecha_fin, court_name)

              start_dt              end_dt
1  2025-07-08 20:30:00 2025-07-08 22:00:00
2  2025-07-08 19:00:00 2025-07-08 20:30:00
3  2025-07-08 17:30:00 2025-07-08 19:00:00
4  2025-07-08 16:00:00 2025-07-08 17:00:00
5  2025-07-08 14:30:00 2025-07-08 16:00:00
6  2025-07-08 13:30:00 2025-07-08 14:30:00
11 2025-07-09 20:30:00 2025-07-09 22:00:00
12 2025-07-09 19:00:00 2025-07-09 20:30:00
13 2025-07-09 18:00:00 2025-07-09 19:00:00
14 2025-07-09 17:00:00 2025-07-09 18:00:00
15 2025-07-09 16:00:00 2025-07-09 17:00:00
16 2025-07-10 20:30:00 2025-07-10 22:00:00
18 2025-07-10 19:00:00 2025-07-10 20:30:00
20 2025-07-10 18:00:00 2025-07-10 19:00:00
21 2025-07-10 16:30:00 2025-07-10 17:30:00
22 2025-07-10 08:30:00 2025-07-10 09:00:00
23 2025-07-11 20:30:00 2025-07-11 22:00:00
25 2025-07-11 19:00:00 2025-07-11 20:30:00
26 2025-07-11 08:30:00 2025-07-11 10:00:00
28 2025-07-12 10:00:00 2025-07-12 11:30:00
False
No existe ninguna reserva para este dia y esta hora
